<a href="https://colab.research.google.com/github/ayushmanlohani/Neural-translator-eng-fr-/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import re

# ==========================================
# 1. YOUR EXACT MODEL ARCHITECTURE
# (Extracted directly from your notebook)
# ==========================================

class TransformerConfig:
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        block_size=128,
        n_layer=6,
        n_pre_cross_layer=3,
        n_cross_layer=3,
        n_embd=256,
        num_heads=8,
        dropout=0.1
    ):
        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size
        self.block_size = block_size
        self.n_layer = n_layer
        self.n_pre_cross_layer = n_pre_cross_layer
        self.n_cross_layer = n_cross_layer
        self.n_embd = n_embd
        self.num_heads = num_heads
        self.dropout = dropout

class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.ReLU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        return self.net(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.num_heads == 0
        self.num_heads = config.num_heads
        self.head_size = config.n_embd // config.num_heads
        self.n_embd = config.n_embd

        self.q_proj = nn.Linear(config.n_embd, config.n_embd)
        self.k_proj = nn.Linear(config.n_embd, config.n_embd)
        self.v_proj = nn.Linear(config.n_embd, config.n_embd)
        self.out_proj = nn.Linear(config.n_embd, config.n_embd)

        self.dropout = nn.Dropout(config.dropout)

    def forward(self, q, k=None, v=None, mask=None, is_causal=False):
        batch_size = q.size(0)
        if k is None: k = q
        if v is None: v = q

        q_out = self.q_proj(q)
        k_out = self.k_proj(k)
        v_out = self.v_proj(v)

        q_out = q_out.view(batch_size, -1, self.num_heads, self.head_size).transpose(1, 2)
        k_out = k_out.view(batch_size, -1, self.num_heads, self.head_size).transpose(1, 2)
        v_out = v_out.view(batch_size, -1, self.num_heads, self.head_size).transpose(1, 2)

        scores = (q_out @ k_out.transpose(-2, -1)) / math.sqrt(self.head_size)

        if is_causal:
            seq_len = q_out.size(-2)
            causal_mask = torch.triu(torch.ones(seq_len, seq_len, dtype=torch.bool, device=q.device), diagonal=1)
            scores.masked_fill_(causal_mask, float('-inf'))

        if mask is not None:
            if mask.dim() == 3: mask = mask.unsqueeze(1)
            scores.masked_fill_(~mask, float('-inf'))

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = attn @ v_out
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.n_embd)
        out = self.out_proj(out)
        return out

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attn = MultiHeadAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.ffwd = FeedForward(config)

    def forward(self, x, mask=None, is_causal=False):
        x = x + self.attn(self.ln1(x), mask=mask, is_causal=is_causal)
        x = x + self.ffwd(self.ln2(x))
        return x

class CrossAttentionBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.self_attn = MultiHeadAttention(config)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.cross_attn = MultiHeadAttention(config)
        self.ln3 = nn.LayerNorm(config.n_embd)
        self.ffwd = FeedForward(config)

    def forward(self, x, enc_out, self_mask=None, cross_mask=None):
        x = x + self.self_attn(self.ln1(x), mask=self_mask, is_causal=True)
        x = x + self.cross_attn(q=self.ln2(x), k=enc_out, v=enc_out, mask=cross_mask)
        x = x + self.ffwd(self.ln3(x))
        return x

class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.blocks = nn.ModuleList([Block(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)

    def forward(self, x, mask=None):
        for block in self.blocks:
            x = block(x, mask=mask)
        return self.ln_f(x)

class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.pre_blocks = nn.ModuleList([Block(config) for _ in range(config.n_pre_cross_layer)])
        self.cross_blocks = nn.ModuleList([CrossAttentionBlock(config) for _ in range(config.n_cross_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)

    def forward(self, x, enc_out, padding_mask=None, cross_mask=None):
        for block in self.pre_blocks:
            x = block(x, mask=padding_mask, is_causal=True)
        for block in self.cross_blocks:
            x = block(x, enc_out, self_mask=padding_mask, cross_mask=cross_mask)
        return self.ln_f(x)

class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.src_tok_emb = nn.Embedding(config.src_vocab_size, config.n_embd)
        self.tgt_tok_emb = nn.Embedding(config.tgt_vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.dropout)
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)
        self.head = nn.Linear(config.n_embd, config.tgt_vocab_size, bias=False)

    def forward(self, src_ids, tgt_ids, src_mask=None, tgt_mask=None):
        B, T_src = src_ids.size()
        _, T_tgt = tgt_ids.size()
        src_emb = self.src_tok_emb(src_ids)
        src_pos = self.pos_emb[:, :T_src, :]
        x = self.drop(src_emb + src_pos)
        encoder_out = self.encoder(x, src_mask)
        tgt_emb = self.tgt_tok_emb(tgt_ids)
        tgt_pos = self.pos_emb[:, :T_tgt, :]
        y = self.drop(tgt_emb + tgt_pos)
        y = self.decoder(y, encoder_out, padding_mask=tgt_mask, cross_mask=src_mask)
        logits = self.head(y)
        return logits

# ==========================================
# 2. HELPER FUNCTIONS & LOADING
# ==========================================

def basic_tokenize(text):
    """Exact tokenizer from your notebook"""
    text = text.lower()
    text = re.sub(r'([.,!?;])', r' \1 ', text)
    text = re.sub(r'([\"\'])', r' \1 ', text)
    text = re.sub(r'[^a-z0-9.,!?;\'\" ]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text.split()

def load_vocab(file_path):
    vocab = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                vocab[parts[0]] = int(parts[1])
    return vocab

def translate_sentence(model, sentence, src_vocab, tgt_vocab, device, max_length=50):
    model.eval()
    tokens = basic_tokenize(sentence)

    # Mapping based on your notebook
    # <pad>:0, <unk>:1, <sos>:2, <eos>:3
    src_indices = [src_vocab.get('<sos>', 2)] + \
                  [src_vocab.get(t, src_vocab.get('<unk>', 1)) for t in tokens] + \
                  [src_vocab.get('<eos>', 3)]

    src_tensor = torch.LongTensor(src_indices).unsqueeze(0).to(device)
    src_mask = (src_tensor != 0).unsqueeze(1).unsqueeze(2)

    with torch.no_grad():
        src_emb = model.src_tok_emb(src_tensor)
        src_pos = model.pos_emb[:, :src_tensor.size(1), :]
        encoder_out = model.encoder(model.drop(src_emb + src_pos), src_mask)

    tgt_indices = [tgt_vocab.get('<sos>', 2)]

    for _ in range(max_length):
        tgt_tensor = torch.LongTensor(tgt_indices).unsqueeze(0).to(device)
        tgt_mask = (tgt_tensor != 0).unsqueeze(1).unsqueeze(2)

        with torch.no_grad():
            tgt_emb = model.tgt_tok_emb(tgt_tensor)
            tgt_pos = model.pos_emb[:, :tgt_tensor.size(1), :]
            y = model.drop(tgt_emb + tgt_pos)

            output = model.decoder(y, encoder_out, padding_mask=tgt_mask, cross_mask=src_mask)
            logits = model.head(output)
            next_token_id = logits[0, -1, :].argmax().item()

            tgt_indices.append(next_token_id)
            if next_token_id == tgt_vocab.get('<eos>', 3):
                break

    idx_to_word = {v: k for k, v in tgt_vocab.items()}
    translated_tokens = []
    for idx in tgt_indices:
        token = idx_to_word.get(idx, '')
        if token not in ['<sos>', '<eos>', '<pad>']:
            translated_tokens.append(token)

    return " ".join(translated_tokens)

# ==========================================
# 3. STREAMLIT APP
# ==========================================

st.set_page_config(page_title="Eng-Fr Translator", layout="centered")

st.title("🇫🇷 English to French Translator")
st.write("Using a custom trained Transformer model.")

@st.cache_resource
def load_resources():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Vocabs
    eng_vocab = load_vocab('eng_vocab.txt')
    fr_vocab = load_vocab('fr_vocab.txt')

    # Initialize Configuration (Using your EXACT notebook params)
    config = TransformerConfig(
        src_vocab_size=len(eng_vocab),
        tgt_vocab_size=len(fr_vocab),
        block_size=128,
        n_layer=6,
        n_pre_cross_layer=3,
        n_cross_layer=3,
        n_embd=256,
        num_heads=8,
        dropout=0.1
    )

    # Load Model
    model = Transformer(config).to(device)
    # Load State Dict
    try:
        model.load_state_dict(torch.load('best_model.pt', map_location=device))
    except Exception as e:
        st.error(f"Failed to load weights: {e}")
        return None, None, None, None

    return model, eng_vocab, fr_vocab, device

model, eng_vocab, fr_vocab, device = load_resources()

if model:
    user_input = st.text_area("Enter English text:", height=100)

    if st.button("Translate"):
        if user_input:
            with st.spinner("Translating..."):
                translation = translate_sentence(model, user_input, eng_vocab, fr_vocab, device)
                st.success("French Translation:")
                st.info(translation)
        else:
            st.warning("Please enter some text.")

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Fetch the token securely
ngrok_token = userdata.get('NGROK_TOKEN')
ngrok.set_auth_token(ngrok_token)

# Terminate open tunnels if any exist
ngrok.kill()

# Run Streamlit in background
!streamlit run app.py &>/dev/null&

# Open the tunnel
try:
    public_url = ngrok.connect(8501).public_url
    print(f"🚀 Click here to use your app: {public_url}")
except Exception as e:
    print(f"Error starting tunnel: {e}")